In [23]:
import pandas as pd
import nltk
import numpy as np
import string
import warnings
import requests
import pickle
import random
#Libreria de coneccion a bd
import mysql.connector
#Libreria de coneccion a bd

nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Veliz\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Veliz\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Veliz\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [25]:
data = [["Hola",0],["Hola, ¿como estas?",0],["Que tal, buenos dias",0],["Buen dia.",0],["Hey, ¿Que tal?",0],["¡Hey!.",0],["Buenas.",0],
        ["¿Que destinos tiene?",1],["¿Que viajes tiene?",1],["¿Que viajes pueden realizar?",1],["¿Pueden mencionarme sus viajes?",1],["En que parte veo sus destinos",1],["Viajan para La merced?",1],["Viajan para La Satipo?",1],["Viajan para La Pichanaki?",1],["Viajan para La Huacayo?",1],["Que destinos pueden ir?",1],["Que destinos puedo ir?",1],["Que viajes tienen?",1],["Me puede mencionar los pasajes?",1],["¨Puede usted mencionarme los pasajes que tienen?",1],["Que destinos cuentan por el momento?",1],
        ["¿Que horarios son sus viajes?",2],["¿Que horarios disponen?",2],["¿Que horarios tienen para viajar?",2],["¿Pueden mencionarme sus horarios de viaje?",2],["¿Me brinda los horarios que dispone para viajar?",2],["Que horarios disponen en estos momentos",2],["Tienen horarios disponibles?",2],["Puede brindarme horarios?",2],["Como puedo saber los horarios?",2],["Me gustaria saber los horarios que tienen disponibles",2],      
        ["¿En que parte se ubican los terminales?",3],["¿Dirección de sus terminales?",3],["En satipo en que lugar se encuentra su terminal?",3],["Estoy en la mercerd, en que lugar esta su terminal?",3],["Se ubican por huancayo?",3],["Que ubicación esta su terminal",3],["Donde esta ubicado su terminal",3],["Su terminal en que parte esta?",3],["En que lugar es su terminal para comprar pasajes?",3],["Tiene varias agencias para la compra de mis pasajes?",3],["En que lugar esta ubicado su agencia?",3],["Su agencia de donde estan?",3],["Agencia?",3],["Terminal?",3],
        ["Cuanto están los pasajes?",4],["Cuento cuestan los pasajes de su empresa?",4],["Cuanto cuestan los pasajes?",4],["Los costos de los pasajes son variados?",4],["Cual es el precio de sus pasajes?",4],["El precio de los pasajes?",4],["Cual es el costo de sus pasajes?",4],["Me podria mencionar sus precios",4],["Costo de pasajes",4],["Precio de pasajes",4],["Costo",4],["Precio",4],
        ["Sabe usted algun lugar turistico que me pueda recomendar?",5],["Lugares turisticos?",5],["Que destinos turisticos me recomienda?",5],["Me podria recomendar un lugar de paseo?",5],["Usted sabe lugares de paseo?",5],["Que lugares de paseo tienen",5],["Paseo",5],["Turistico",5],["Turismo",5],["Lugares turisticos",5],["Paseos familiares",5],
        ["Como puedo comprar un viaje?",6],["Puedo comprar mi pasaje con tarjeta?",6],["Que metodos de pago tiene?",6],["Con que trajetas puedo pagar?",6],["Aceptan efectivo?",6],["Puedo comprar mi pasaje con otro metodo?",6],["Tiene variedad de metodos?",6],["Métodos?",6],["Métodos de pago",6],["Como puedo pagar",6],["Pago",6],["Pagar",6],["Comprar",6],
        ["Que asientos tiene disponible?",7],["Los asientos disponibles que tienes cuales son?",7],["Dispone de asientos disponibles?",7],["Asientos",7],["Los asientos que cuenta?",7],["Cuales asientos le resta",7],
        ["Cuales son tus políticas de la empresa",8],["La empresa cuenta son políticas para reclamos?",8],["políticas",8],["política",8],["Dónde puedo ver que políticas tiene la empresa?",8],
        ["Me gustaría hacer una compra de pasaje",9],["Comprar pasaje",9],["Adquirir la compra del pasaje",9],
        ["Nos vemos",10],["Hasta luego",10],["Nos vemos pronto",10],["Hasta pronto",10],["Nos vemos luego",10],["Bye",10],["hasta la proxima",10],
        ["Muchas gracias por su atencion",11],["Gracias, hasta luego",11],["Gracias por su atención",11],["Gracias!",11],["buen servicio, hasta pronto",11]]
df = pd.DataFrame(data, columns = ["Text","Intent"])
df

,Text,Intent
0,Hola,0
1,"Hola, ¿como estas?",0
2,"Que tal, buenos dias",0
3,Buen dia.,0
4,"Hey, ¿Que tal?",0
...,...,...
103,Muchas gracias por su atencion,11
104,"Gracias, hasta luego",11
105,Gracias por su atención,11
106,Gracias!,11


In [26]:
# Lemmitization

lemmer = nltk.stem.WordNetLemmatizer()

In [27]:
def LemTokens(tokens):
    return [lemmer.lemmatize(token) for token in tokens]

In [28]:
remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)

In [29]:
def Normalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

In [30]:
x = df['Text']
y = df['Intent']

In [31]:
vectorizer = TfidfVectorizer(tokenizer=Normalize,stop_words = 'english')

In [32]:
X = vectorizer.fit_transform(x)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


In [33]:
# training a Naive Bayes classifier 
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()

lr.fit(X, y)

LogisticRegression()

In [34]:
X_test = ["flowers"]
prediction = lr.predict(vectorizer.transform(X_test))

In [35]:
prediction

array([1], dtype=int64)

In [36]:
# To get the probability of test data to be present in each Intent

a = lr.predict_proba(vectorizer.transform(X_test))
a

array([[0.08025033, 0.13048288, 0.08358773, 0.11751616, 0.1005759 ,
        0.10573135, 0.12706034, 0.05366456, 0.05028101, 0.03208757,
        0.06926634, 0.04949584]])

In [30]:
db = mysql.connector.connect(
  host="MYSQL5027.site4now.net",
  user="a7b711_busproy",
  password="Sistemas10",
  port= "3306",  
  database= "db_a7b711_busproy"
)

cursor = db.cursor()
cursor.execute("SELECT codigo FROM minivan  WHERE id = 1")
vista = str(cursor.fetchall()[0:])
print(vista)
inicio = vista.find("'")
print(inicio)
final = vista.find("'",inicio+1)
print(final)
extraerdato=vista[inicio+1:final]
print(extraerdato)

[('CARRITO1',)]
2
11
CARRITO1


In [37]:
db = mysql.connector.connect(
  host="MYSQL5027.site4now.net",
  user="a7b711_busproy",
  password="Sistemas10",
  port= "3306",  
  database= "db_a7b711_busproy"
)

cursor = db.cursor()
cursor.execute("SELECT hora_salida FROM viaje_detalle where id=1")
vista2 = str(cursor.fetchall()[0:])
inicio = vista2.find("=")
final = vista2.find(")")
extraerdato=int(vista2[inicio+1:final])
print(extraerdato)

35100


In [38]:
import time
from datetime import datetime, timedelta
def convert_to_hours(delta):
    total_seconds = delta.total_seconds()
    hours = str(int(total_seconds // 3600)).zfill(2)
    minutes = str(int((total_seconds % 3600) // 60)).zfill(2)
    seconds = str(int(total_seconds % 60)).zfill(2)
    return f"{hours}:{minutes}:{seconds}"

delta = timedelta(seconds=extraerdato)
Finalextraido=convert_to_hours(delta)
HoraFinal=Finalextraido+" AM"
print(HoraFinal)

09:45:00 AM


In [39]:
db = mysql.connector.connect(
  host="MYSQL5027.site4now.net",
  user="a7b711_busproy",
  password="Sistemas10",
  port= "3306",  
  database= "db_a7b711_busproy"
)

cursor = db.cursor()
cursor.execute("SELECT precio_viaje FROM viaje_detalle where id=1")
vista6a = str(cursor.fetchall()[0:])
inicio = vista6a.find("'")
final = vista6a.find("'",inicio+1)
extraerdato=vista6a[inicio+1:final]
PagFin1="S/. "+extraerdato+", "
print(PagFin1)

S/. 30.00


In [42]:
db = mysql.connector.connect(
  host="MYSQL5027.site4now.net",
  user="a7b711_busproy",
  password="Sistemas10",
  port= "3306",  
  database= "db_a7b711_busproy"
)

cursor = db.cursor()
cursor.execute("SELECT precio_viaje FROM viaje_detalle where id=2")
vista6b = str(cursor.fetchall()[0:])
inicio = vista6b.find("'")
final = vista6b.find("'",inicio+1)
extraerdato=vista6b[inicio+1:final]
PagFin2=", S/. "+extraerdato
print(PagFin2)

S/. 15.00


In [43]:
db = mysql.connector.connect(
  host="MYSQL5027.site4now.net",
  user="a7b711_busproy",
  password="Sistemas10",
  port= "3306",  
  database= "db_a7b711_busproy"
)

cursor = db.cursor()
cursor.execute("SELECT precio_viaje FROM viaje_detalle where id=3")
vista6c = str(cursor.fetchall()[0:])
inicio = vista6c.find("'")
final = vista6c.find("'",inicio+1)
extraerdato=vista6c[inicio+1:final]
PagFin3=", S/. "+extraerdato
print(PagFin3)

S/. 20.00


In [44]:
db = mysql.connector.connect(
  host="MYSQL5027.site4now.net",
  user="a7b711_busproy",
  password="Sistemas10",
  port= "3306",  
  database= "db_a7b711_busproy"
)

cursor = db.cursor()
cursor.execute("SELECT num_asiento FROM asiento_usuario_detalle where id = 1")
vista7a = str(cursor.fetchall()[0:])
inicio = vista7a.find("(")
final = vista7a.find(",",inicio+1)
extraerdato=vista7a[inicio+1:final]
AsitDis1="Primer minivan: "+extraerdato
print(AsitDis1)

Primer minivan: 2


In [45]:
db = mysql.connector.connect(
  host="MYSQL5027.site4now.net",
  user="a7b711_busproy",
  password="Sistemas10",
  port= "3306",  
  database= "db_a7b711_busproy"
)

cursor = db.cursor()
cursor.execute("SELECT num_asiento FROM asiento_usuario_detalle where id = 2")
vista7b = str(cursor.fetchall()[0:])
inicio = vista7b.find("(")
final = vista7b.find(",",inicio+1)
extraerdato=vista7b[inicio+1:final]
AsitDis2=", Segunda minivan: "+extraerdato
print(AsitDis2)

, Segunda minivan: 1


In [46]:
responses = {0 : {"intent":"greetings","response":['Buen día, ¿En que le puedo ayudar?','Hola, ¿Cómo le va?','Hola, un gusto tenerte por aquí.',]}, 
      1 : {"intent":"availability","response":['Sí, nuestros viajes por el momento son hacia: Huancayo, La Merced, Pichanaki y Satipo','Claro, tenemos viajes con destinos a: Huancayo, La Merced, Pichanaki y Satipo']},
      2 : {"intent":"flowers","response":['Los horarios que tenemos disponibles son: '+HoraFinal, 'Tenemos los siguentes horarios disponibles'+HoraFinal]},
      3 : {"intent":"flowers","response":['Nos ubicamos en Jr.Aviación 291 en Satipo, Av. Marginal 294 en Pichanaki, Av. Carlos A. En La Merced, Terminal terrestre en huancayo','Estamos ubicados en Jr.Aviación 291 en Satipo, Av. Marginal 294 en Pichanaki, Av. Carlos A. En La Merced, Terminal terrestre en huancayo']},
      4 : {"intent":"flowers","response":["Los precios disponibles que tenemos son: "+PagFin1+PagFin2+PagFin3, 'Nuestros precios son: '+PagFin1+PagFin2+PagFin3]},
      5 : {"intent":"flowers","response":['Las cuidades a donde vamos tienen varios destinos turisticos que puede visitar.']},
      6 : {"intent":"flowers","response":['Se puede pagar con Paypal','Puede usted realizar los pagos con Paypal','Los pagos se realizan con Paypal']},
      7 : {"intent":"flowers","response":['Estos son nuestros asientos disponibles'+AsitDis1+AsitDis2, 'Te mostramos los asientos disponibles'+AsitDis1+AsitDis2]},
      8 : {"intent":"flowers","response":['Se mostrará nuestra políticas de trabajo a continuación: ','Claro, se le proporcionará nuestras políticas: ','Con mucho gusto, le mostraré un formato donde puede leer toda nuestra información: ']},
      9 : {"intent":"flowers","response":['Puede realizar su compra haciendo click en este bot: @.','Usted puede realizar la compra de su pasaje haciendo click en el siguente bot: @']},
      10 : {"intent":"goodbye","response":['Vuelva pronto.','Hasta pronto.','Chau, nos vemos pronto','Ten un buen día.']},
      11 : {"intent":"thankyou","response":['No se preocupe.' , 'No hay problema.', 'No hay problema, vuelva pronto.']}}

In [47]:
import json
token = "1984252857:AAFMGeqeiuVp_318l487D-wy4jfJFPxp2Fw"

class telegram_bot():
    def __init__(self):
        self.token=token 
        self.url = f"https://api.telegram.org/bot{self.token}"

    def get_updates(self,offset=None):
        url = self.url+"/getUpdates?timeout=100"
        if offset:
            url = url+f"&offset={offset+1}"
        url_info = requests.get(url)
        return json.loads(url_info.content)
    def send_message(self,msg,chat_id):
        url = self.url + f"/sendMessage?chat_id={chat_id}&text={msg}"
        if msg is not None:
            requests.get(url)

    def grab_token(self):
        return tokens

In [48]:
# To get responnse

def response(user_response):
    text_test = [user_response]
    X_test = vectorizer.transform(text_test)
    prediction = lr.predict(X_test)
    reply = random.choice(responses[prediction[0]]['response'])
    return reply

# To get indent
def intent(user_response):
    text_intent = [user_response]
    X_test_intent = vectorizer.transform(text_intent)
    predicted_intent = lr.predict(X_test_intent)
    intent_predicted = responses[predicted_intent[0]]['intent']
    return intent_predicted

In [49]:
import random

def bot_initialize(user_msg):
    flag=True
    while(flag==True):
        user_response = user_msg
        
        user_intent = intent(user_response)
        
        if(user_intent != 'goodbye'):
            if(user_response == '/start'):
                resp = """Bienvenido a servicios Turismo SHIMA, aquí podemos responder algunas de tus consultas que tengas."""
                return resp
            
            elif (user_intent == 'greetings'):
                resp = str(random.choice(responses[0]['response'])) + ", ¿Como puedo ayudarte?"
                return resp
        
            elif(user_intent == 'thankyou'):
                resp = random.choice(responses[11]['response'])
                return resp
            
            elif(user_intent == 'availability'):
                resp = random.choice(responses[1]['response'])
                return resp
            
            elif(user_intent == "flowers"):
                user_response=user_response.lower()
                resp = "Posible respuesta:-\n  " + response(user_response)
                return resp
            
            else:
                resp = "Mmmm! Podría ser más especifico por favor."
                return resp
            
        else:
            flag = False
            resp = random.choice(responses[9]['response'])
            return resp

In [50]:
tbot = telegram_bot()

update_id = None

def make_reply(msg):
    if msg is not None:
        reply = bot_initialize(msg)
    return reply


while True:
    print("...")
    updates = tbot.get_updates(offset=update_id)
    updates = updates['result']
    print(updates)
    if updates:
        for item in updates:
            update_id = item["update_id"]
            print(update_id)
            try:
                message = item["message"]["text"]
                print(message)
            except:
                message = None
            from_ = item["message"]["from"]["id"]
            print(from_)

            reply = make_reply(message)
            tbot.send_message(reply,from_)

...
[{'update_id': 131167067, 'message': {'message_id': 395, 'from': {'id': 1918631995, 'is_bot': False, 'first_name': 'Miguel', 'last_name': 'Veliz', 'language_code': 'en'}, 'chat': {'id': 1918631995, 'first_name': 'Miguel', 'last_name': 'Veliz', 'type': 'private'}, 'date': 1637772239, 'text': '/start', 'entities': [{'offset': 0, 'length': 6, 'type': 'bot_command'}]}}]
131167067
/start
1918631995
...
[{'update_id': 131167068, 'message': {'message_id': 397, 'from': {'id': 1918631995, 'is_bot': False, 'first_name': 'Miguel', 'last_name': 'Veliz', 'language_code': 'en'}, 'chat': {'id': 1918631995, 'first_name': 'Miguel', 'last_name': 'Veliz', 'type': 'private'}, 'date': 1637772255, 'text': 'precios de los pasajes'}}]
131167068
precios de los pasajes
1918631995
...
[{'update_id': 131167069, 'message': {'message_id': 399, 'from': {'id': 1918631995, 'is_bot': False, 'first_name': 'Miguel', 'last_name': 'Veliz', 'language_code': 'en'}, 'chat': {'id': 1918631995, 'first_name': 'Miguel', 'last

ConnectionError: HTTPSConnectionPool(host='api.telegram.org', port=443): Max retries exceeded with url: /bot1984252857:AAFMGeqeiuVp_318l487D-wy4jfJFPxp2Fw/sendMessage?chat_id=1918631995&text=Bienvenido%20a%20servicios%20Turismo%20SHIMA,%20aqu%C3%AD%20podemos%20responder%20algunas%20de%20tus%20consultas%20que%20tengas. (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000002A901DC0790>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))